# Biomphalaria organ microbiome


## Aim

Lauren Carruthers internship project. Look at microbiomes of different tissues from two *Biomphalaria* species.


Several folders present or will be created during the analysis. Here is the list of the folders and their content:
* **data**: Raw data files used for the analysis.
* **.env**: Files needed to create appropriate environment.
* **graphs**: Graphical representation of the data. If not existing, this will be created during the analysis
* **results**: Files that are generating through data processing. If not existing, this will be created during the analysis
* **scripts**: Scripts used for the analysis.


## Environment and data

### Creating environment

In [ ]:
# Check if conda available
[[ ! $(which conda 2> /dev/null) ]] && echo "conda not available in \$PATH. Exiting..." && exit 1

# Creating conda environment
conda env create -f .env/env.yml

This cell must be run each time a new session of Jupyter is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate ubiome_organs

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Downloading sequencing data

This step downloads the fastq files of the different samples.

**vvvv To be removed** This generates the data and will have to be replaced by the downloading of the data from SRA

In [ ]:
# Project directory
cd $HOME/analyses/11-Microbiome/
mkdir 2019-08-08_Biomphalaria_tissues
cd 2019-08-08_Biomphalaria_tissues

# Fasta generation (MiSeq Output previously uploaded)
cd 0-Raw\ data/190806_M01370_0001_000000000-CKY27/
nohup bcl2fastq --output-dir ../fastq_files/
cd ../..

# Working directory
mkdir 1-Qiime
cd 1-Qiime

# Link data
mkdir data
ln -s ../0-Libraries/*.fastq.gz data/

**^^^ To be removed**

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=PRJNXXXXXXX

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    #wget -P "$ldir" -O "$fln" "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?cmd=dload&run_list=${run}&format=fastq"
    fastq-dump -O "$ldir" --split-files "$run"
    
    mv "$ldir/${run}_1.fastq" "$ldir/${fln}_R1.fastq"
    mv "$ldir/${run}_2.fastq" "$ldir/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | head -1)

# Compress files
pigz "$ldir/"*

rm runinfo

### Downloading database

The Silva database is used to assign taxonomy to the ASVs generated from the sequencing data.

In [ ]:
# Database directory
dbdir="data/Silva db"
[[ ! -d "$dbdir" ]] && mkdir -p "$dbdir"


# Download and extract the relevant Silva file
wget -P "$dbdir" 'https://www.arb-silva.de/fileadmin/silva_databases/qiime/Silva_132_release.zip'
unzip "$dbdir/Silva_132_release.zip" -d "$dbdir" && rm "$dbdir/Silva_132_release.zip"

# Import the sequence database in Qiime format
qiime tools import \
  --input-path "$dbdir/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna" \
  --output-path "$dbdir/silva_132_99_16S.qza" \
  --type 'FeatureData[Sequence]'

# Import the taxonomy database in Qiime format
qiime tools import \
  --input-path "$dbdir/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt" \
  --output-path "$dbdir/silva_132_99_16S_taxa.qza" \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat

## Qiime pipeline

This section process the data to generate ASVs and assign taxonomy.

In [ ]:
# Qiime output directory
qdir="results/1-qiime"
[[ ! -d "$qdir" ]] && mkdir -p "$qdir"

# Metadata file
metadata="data/sample-metadata.tsv"

**vvvv To be removed** This generates the manifest from fastq generated by bcl2fastq and will have to be replaced by the cell after that generates the manifest from the downloaded SRA data

In [5]:
# Create the manifest for importing data in artefact
## source: https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats
for i in $(ls data/* | cut -d "_" -f 3-5 | uniq)
do
    nm=$(sed "s,data/,, ; s,_,.,g" <<<$i)
    fl=$(ls -1 $PWD/data/*$i* | tr "\n" "\t")

    echo -e "$nm\t$fl"
done > "$qdir/manifest"

# Add header
sed -i "1s/^/sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n/" "$qdir/manifest"

# Import data
## source: https://docs.qiime2.org/2019.4/tutorials/importing/
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path "$qdir/manifest" \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path "$qdir/demux-paired-end.qza"

**^^^ To be removed**

In [ ]:
# Check for sequencing data
[[ ! $(find "$ldir" -type f -name *fastq.gz) ]] && echo  "No sequencing data. Exiting..." && exit 1

# Create the manifest for importing data in artefact
## source: https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats
for i in $(ls "$ldir"/* | sed "s,_R[12].fastq.*,,g" | uniq)
do
    nm=$(sed "s,$ldir/,," <<<$i)
    fl=$(ls -1 "$PWD/$i"* | tr "\n" "\t")

    echo -e "$nm\t$fl"
done > "$qdir/manifest"

# Add header
sed -i "1s/^/sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n/" "$qdir/manifest"

# Import data
## source: https://docs.qiime2.org/2019.4/tutorials/importing/
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path "$qdir/manifest" \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path "$qdir/demux-paired-end.qza"

**Note**:
* No need to remove adapters and barcodes. This has been done during `bcl2fastq`. This can be checked using `grep`.
* Importing with `--input-format PairedEndFastqManifestPhred33V2` instead of `--input-format CasavaOneEightSingleLanePerSampleDirFmt` for [custom sample names](https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats).

### Data quality

To assess data quality, we need to generate a visualization to check data quality. The visualization can be view on [Qiime2 website](https://view.qiime2.org/)

In [9]:
# Make a summary to check read quality
qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

(qiime2-2019.4) Saved Visualization to: demux-paired-end2.qzv
(qiime2-2019.4) 

: 1

**Note**: read quality drops toward the end but are still above 10. So no trimming done.

### Sequence clustering and denoising

This steps generates ASVs from the sequencing data. This step is perform by the `dada2` module.

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs "$qdir/demux-paired-end.qza" \
  --p-trunc-len-f 177 \
  --p-trunc-len-r 202 \
  --p-trim-left-f 0 \
  --p-trim-left-r 13 \
  --p-max-ee-f 5 \
  --p-max-ee-r 10 \
  --p-n-threads 0 \
  --o-table "$qdir/table.qza" \
  --o-representative-sequences "$qdir/rep-seqs.qza" \
  --o-denoising-stats "$qdir/denoising-stats.qza"

In [ ]:
qiime feature-classifier classify-consensus-vsearch \
  --i-query "$qdir/rep-seqs.qza" \
  --i-reference-reads database/silva_132_99_16S.qza \
  --i-reference-taxonomy database/silva_132_99_16S_taxa.qza \
  --p-perc-identity 0.97 \
  --p-threads $(nproc) \
  --o-classification "$qdir/rep-seqs_taxa.qza"

In [ ]:
# source: https://chmi-sops.github.io/mydoc_qiime2.html

# Multiple seqeunce alignment using Mafft
qiime alignment mafft \
    --i-sequences "$qdir/rep-seqs.qza" \
    --o-alignment "$qdir/aligned-rep-seqs.qza"

# Masking (or filtering) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree.
qiime alignment mask \
    --i-alignment "$qdir/aligned-rep-seqs.qza" \
    --o-masked-alignment "$qdir/masked-aligned-rep-seqs.qza"

# Creating tree using the Fasttree program
qiime phylogeny fasttree \
    --i-alignment "$qdir/masked-aligned-rep-seqs.qza" \
    --o-tree "$qdir/unrooted-tree.qza"

# Root the tree using the longest root
qiime phylogeny midpoint-root \
    --i-tree "$qdir/unrooted-tree.qza" \
    --o-rooted-tree "$qdir/rooted-tree.qza"

## Trunk

### Sample metadata updates

In [ ]:
# Create the manifest for importing data in artefact
## source: https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats
for i in $(ls data/* | cut -d "_" -f -5 | uniq)
do
    nm=$(sed "s,data/,, ; s,_,.,g" <<<$i)
    cln=$(echo "$nm" | cut -d "." -f 4)
    cln="$cln\t$(echo "$nm" | cut -d "." -f 4-5)"
    cln="$cln\t$(echo "$nm" | cut -d "." -f 5)"
    
    # Update name
    nm=$(echo "$nm" | cut -d "." -f 3-5)

    echo -e "$nm\t$cln"
done > sample-metadata.tsv

# Add header
sed -i "1s/^/sample-id\tSpecies\tComb\tTissue\n/" sample-metadata.tsv

In [19]:
## !! WARNING To be removed when samplesheet corrected
sed -i "s/\tCa/\tBa/g" sample-metadata.tsv

(qiime2-2019.4) (qiime2-2019.4) (qiime2-2019.4) (qiime2-2019.4) (qiime2-2019.4) (qiime2-2019.4) 

: 1